In [58]:
import scanpy as sc
from anndata import AnnData
import polars as pl
data = sc.read("data/pbmc3k_pped.h5ad")


In [59]:
data.obsm.keys()

KeysView(AxisArrays with keys: X_pca, X_tsne, X_umap)

In [60]:
def _decide_dimensions_key(data: AnnData, dimensions: str) -> str:
    """Decide on which key to use for the dimensions."""
    if isinstance(data, AnnData):
        # get every key that contains the name of the dimensions
        keys_list = [key for key in data.obsm if dimensions in key.lower()]
        if len(keys_list) == 0:
            msg = f"dimensions '{dimensions}' not found in the data"
            raise KeyError(msg)
        elif len(keys_list) == 1:
            dimensions_key = keys_list[0]
        else:  # multiple keys found
            # shorter keys have precedence
            max_len = max(len(key) for key in keys_list)
            min_len = min(len(key) for key in keys_list)
            # 2d have precedence (over 3d etc.)
            has_2d = [key for key in keys_list if "2d" in key.lower()]
            # resolve the keys
            if max_len > min_len:
                dimensions_key = min(keys_list, key=len)
            elif len(has_2d) > 0:
                dimensions_key = has_2d[0]
            else:
                dimensions_key = keys_list[0]

    return dimensions_key

In [61]:
_decide_dimensions_key(data=data, dimensions="umap")

'X_umap'

In [62]:
keys_list = ["X_umapa", "X_umap2d", "X_umap3d"]
dimensions = "umap"

In [63]:
if len(keys_list) == 0:
    msg = f"dimensions '{dimensions}' not found in the data"
    raise KeyError(msg)
elif len(keys_list) == 1:
    dimensions_key = keys_list[0]
else:  # multiple keys found
    # shorter keys have precedence
    max_len = max(len(key) for key in keys_list)
    min_len = min(len(key) for key in keys_list)
    # 2d have precedence (over 3d etc.)
    has_2d = [key for key in keys_list if "2d" in key.lower()]
    # resolve the keys
    if max_len > min_len:
        dimensions_key = min(keys_list, key=len)
    elif len(has_2d) > 0:
        dimensions_key = has_2d[0]
    else:
        dimensions_key = keys_list[0]

In [64]:
dimensions_key

'X_umapa'

In [66]:
pl.DataFrame().with_columns(pl.from_numpy(data.obsm["X_umap"][:, (0, 1)], schema=["umap1", "umap2"]))

umap1,umap2
f32,f32
16.291706,7.037356
4.026419,3.467973
7.226518,17.208639
2.384201,-2.120665
8.450775,5.837933
…,…
8.15538,5.079872
6.177976,0.910435
2.751972,1.829315
